In [5]:
from dotenv import load_dotenv, find_dotenv
from pprint import pprint
import os
env = load_dotenv(find_dotenv()) # read local .env file
pprint(os.environ.get("OPENAI_API_BASE"))

'https://alkopenai.openai.azure.com/'


In [6]:
from langchain.llms import AzureOpenAI
from langchain.chains import ConversationChain

# first initialize the large language model
llm = AzureOpenAI(
	temperature=0,
	openai_api_version="2023-03-15-preview",
    deployment_name="Davinci003", 
	model_name="text-davinci-003"
)

# now initialize the conversation chain
conversation = ConversationChain(llm=llm)


In [4]:
print(conversation.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [5]:
from langchain.chains.conversation.memory import ConversationBufferMemory

conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

In [6]:
conversation_buf("Good morning AI!")

{'input': 'Good morning AI!',
 'history': '',
 'response': " Good morning! It's a beautiful day today, isn't it? How can I help you?"}

In [7]:
conversation_buf("My name is Gian Maria!")

{'input': 'My name is Gian Maria!',
 'history': "Human: Good morning AI!\nAI:  Good morning! It's a beautiful day today, isn't it? How can I help you?",
 'response': ' Nice to meet you Gian Maria! What can I do for you today?'}

In [8]:
conversation_buf("What is my name and what time is it?")

{'input': 'What is my name and what time is it?',
 'history': "Human: Good morning AI!\nAI:  Good morning! It's a beautiful day today, isn't it? How can I help you?\nHuman: My name is Gian Maria!\nAI:  Nice to meet you Gian Maria! What can I do for you today?",
 'response': ' Your name is Gian Maria and it is currently 10:30am.'}

In [9]:
from langchain.callbacks import get_openai_callback
from pprint import pprint
def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        pprint(f'Spent a total of {cb.total_tokens} tokens')

    return result

In [13]:
from pprint import pprint
result = count_tokens(
    conversation_buf, 
    "My interest here is to explore the potential of integrating Large Language Models with external knowledge"
)
pprint (result)

Spent a total of 555 tokens
(' Yes, that is a great idea! Large Language Models can be used to integrate '
 'external knowledge by using the model to generate responses based on the '
 'external knowledge. For example, you can use the model to identify patterns '
 'in the external knowledge and use those patterns to generate more accurate '
 'responses. Additionally, you can use the model to identify relationships '
 'between different pieces of external knowledge and use those relationships '
 'to generate more meaningful responses.')


In [14]:
pprint(conversation_buf.memory.buffer)

('Human: Good morning AI!\n'
 "AI:  Good morning! It's a beautiful day today, isn't it? How can I help "
 'you?\n'
 'Human: My name is Gian Maria!\n'
 'AI:  Nice to meet you Gian Maria! What can I do for you today?\n'
 'Human: What is my name and what time is it?\n'
 'AI:  Your name is Gian Maria and it is currently 10:30am.\n'
 'Human: My interest here is to explore the potential of integrating Large '
 'Language Models with external knowledge\n'
 'AI:  Interesting! Large Language Models are a type of artificial '
 'intelligence that can process natural language and generate meaningful '
 'responses. They can be used to integrate external knowledge by using the '
 'model to generate responses based on the external knowledge. For example, if '
 'you have a large language model trained on a specific domain, you can use it '
 'to generate responses based on the external knowledge in that domain.\n'
 'Human: My interest here is to explore the potential of integrating Large '
 'Language Mo

In [17]:
from langchain.chains.conversation.memory import ConversationSummaryMemory

conversation_sum = ConversationChain(
	llm=llm,
	memory=ConversationSummaryMemory(llm=llm)
)


Summary memory has a reference to a llm because it will use that llm to summarize the prompt memory

In [18]:
pprint(conversation_sum.memory.prompt.template)

('Progressively summarize the lines of conversation provided, adding onto the '
 'previous summary returning a new summary.\n'
 '\n'
 'EXAMPLE\n'
 'Current summary:\n'
 'The human asks what the AI thinks of artificial intelligence. The AI thinks '
 'artificial intelligence is a force for good.\n'
 '\n'
 'New lines of conversation:\n'
 'Human: Why do you think artificial intelligence is a force for good?\n'
 'AI: Because artificial intelligence will help humans reach their full '
 'potential.\n'
 '\n'
 'New summary:\n'
 'The human asks what the AI thinks of artificial intelligence. The AI thinks '
 'artificial intelligence is a force for good because it will help humans '
 'reach their full potential.\n'
 'END OF EXAMPLE\n'
 '\n'
 'Current summary:\n'
 '{summary}\n'
 '\n'
 'New lines of conversation:\n'
 '{new_lines}\n'
 '\n'
 'New summary:')
